**МОНИТОРИНГ СОЦИАЛЬНЫХ ВЫПЛАТ**

In [81]:
import pandas as pd
from datetime import date
import os
import sys
import shutil # упаковка в архив
import warnings # игнорирование предупреждения о формате при импорте из excel

warnings.simplefilter("ignore")

# функции-хуюнкции
def lat(row):
    if row["r_latitude"] == "no_data":
        if row["d_latitude"] == "no_data":
            return row["mo_latitude"]
        else:
            return row["d_latitude"]
    else:
        return row["r_latitude"]
    
def lon(row):
    if row["r_longtitude"] == "no_data":
        if row["d_longtitude"] == "no_data":
            return row["mo_longtitude"]
        else:
            return row["d_longtitude"]
    else:
        return row["r_longtitude"]
    
def prefix(row):
    if row["r_prefix"] == "no_data":
        if row["d_prefix"] == "no_data":
            return row["mo_prefix"].lower()
        else:
            return row["d_prefix"].lower()
    else:
        return row["r_prefix"].lower()
    
def region(row):
    if row["r_region_name"] == "no_data":
        if row["d_region_name"] == "no_data":
            return row["mo_region_name"].lower()
        else:
            return row["d_region_name"].lower()
    else:
        return row["r_region_name"].lower()
    
def area(row):
    if row["r_area"] == "no_data":
        if row["d_area"] == "no_data":
            return row["mo_area"].lower()
        else:
            return row["d_area"].lower()
    else:
        return row["r_area"].lower()

In [ ]:
# пути к файлам и версии справочников (из названия файлов)
dir_path = os.getcwd()
dir_ = os.listdir(dir_path.replace("\\", "/"))
for file in dir_:
    if file.split("_")[0].split(".")[-1] == "1461":
        path_mo = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            mo_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
    if file.split("_")[0].split(".")[-1] == "114":
        path_dept = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            sp_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
    if file.split("_")[0].split(".")[-1] == "115":
        path_room = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            room_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
    if file.split("_")[0].split(".")[-1] == "1123":
        path_address = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            address_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
# path_save_sp = dir_path.replace("\\", "/") + "/sp.csv"
# path_save_room = dir_path.replace("\\", "/") + "/room.csv"
print("Версии справочников, извлечённые из названий файлов:")
print(
    f"\t- справочник МО: {mo_version}\n\t- справочник СП: {sp_version}\n\t- справочник кабинетов: {room_version}\n\t- справочник адресов: {address_version}"
)
if path_mo.split('.')[-1] == 'zip' or path_dept.split('.')[-2] == 'zip' or path_room.split('.')[-2] == 'zip':
    print('Скачанные архивы не распакованы, программа завершает работу.\nПопробуйте распаковать архивы.')
    sys.exit()

print('Загрузка справочников:')

df_mo = pd.read_excel(
    path_mo, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник МО")
df_dept = pd.read_excel(
    path_dept, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник СП")
df_room = pd.read_excel(
    path_room, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник кабинетов")
df_address = pd.read_excel(
    path_address, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник адресов")

In [231]:
# локальные справочники
df_reg_dict = pd.read_csv("C:/work/scripts/справочники/social_benefits_reg.csv", sep=";")
df_reg_dict["reg_nsi"] = df_reg_dict["reg_nsi"].str.lower().str.replace(r'\W', '', regex=True)
df_reg_dict["reg_rosstat"] = df_reg_dict["reg_rosstat"].str.lower().str.replace(r'\W', '', regex=True)
df_clusters = pd.read_csv("C:/work/scripts/справочники/c&t.csv", sep=";")
df_clusters["region_name"] = df_clusters["region_name"].str.lower().str.replace(r'\W', '', regex=True)
df_clusters["pref"] = df_clusters["pref"].str.lower().str.replace(r'\W', '', regex=True)
df_clusters["area"] = df_clusters["area"].str.lower().str.replace(r'\W', '', regex=True)

In [219]:
# преобразование
df_room_s = (
    df_room[
        [
            "mo_oid",
            "depart_oid",
            "oid",
            "building_address_latitude",
            "building_address_longtitude",
            "building_address_region_name",
            "building_address_prefix_area",
            "building_address_area_name",
        ]
    ]
    .drop(index=0)
    .rename(columns={
        "oid": "room_oid",
        "building_address_latitude": "r_latitude",
        "building_address_longtitude": "r_longtitude",
        "building_address_region_name": "r_region_name",
        "building_address_prefix_area": "r_prefix",
        "building_address_area_name": "r_area",
        })
)
df_dept_s = (
    df_dept[
        [
            "depart_oid",
            "building_address_latitude",
            "building_address_longtitude",
            "building_address_region_name",
            "building_address_prefix_area",
            "building_address_area_name",
        ]
    ]
    .drop(index=0)
    .rename(
        columns={
            "building_address_latitude": "d_latitude",
            "building_address_longtitude": "d_longtitude",
            "building_address_region_name": "d_region_name",
            "building_address_prefix_area": "d_prefix",
            "building_address_area_name": "d_area",
        }
    )
)
df_mo_s = (
    df_mo[
        [
            "oid",
            "regionName",
            "latitude",
            "longtitude",
            "addrRegionName",
            "prefixArea",
            "areaName",
        ]
    ]
    .drop(index=0)
    .rename(columns={
        "oid": "mo_oid",
        "regionName": "mo_reg_name",
        "latitude": "mo_latitude",
        "longtitude": "mo_longtitude",
        "addrRegionName": "mo_region_name",
        "areaName": "mo_area",
        "prefixArea": "mo_prefix",
        })
)

In [233]:
%%time

# формирование адекватного адреса
df_adr = df_room_s.merge(df_mo_s, on="mo_oid", how="left").merge(df_dept_s, on="depart_oid", how="left")
df_adr = df_adr.fillna("no_data")
df_adr["lat"] = df_adr.apply(lat, axis=1)
df_adr["lon"] = df_adr.apply(lon, axis=1)
df_adr["reg"] = df_adr.apply(region, axis=1).str.replace('город ', '').str.replace(r'\W', '', regex=True)
df_adr["pref"] = df_adr.apply(prefix, axis=1).str.replace(r'\W', '', regex=True)
df_adr["area"] = df_adr.apply(area, axis=1).str.replace('город ', '').str.replace(r'\W', '', regex=True)
df_adr_s = df_adr[["mo_oid", "depart_oid", "room_oid", "lat", "lon", "reg", "pref", "area"]]
df_adr_s = df_adr_s.merge(df_reg_dict, left_on="reg", right_on="reg_nsi", how="left")
df_adr_s.drop("reg_nsi", axis=1, inplace=True)
# объединение
df_pclust = df_adr_s.merge(df_clusters, left_on=["reg_rosstat", "area"], right_on=["region_name", "area"], how="left")


CPU times: total: 14.4 s
Wall time: 56.7 s


In [234]:
# df_pclust[~df_pclust["pop_cluster"].isna()]
df_pclust[df_pclust["population"].isna()]["pref_x"].unique()
df_pclust[(df_pclust["pref_x"] == "г") & (df_pclust["pop_cluster"].isna())]

,mo_oid,depart_oid,room_oid,lat,lon,reg,pref_x,area,reg_rosstat,region_name,pref_y,population,pop_cluster
247,1.2.643.5.1.13.13.12.2.77.8335,1.2.643.5.1.13.13.12.2.77.8335.0.117053,1.2.643.5.1.13.13.12.2.77.8335.0.117053.14200,55.826402,37.456828,гмосква,г,троицк,москва,NaN,NaN,NaN,NaN
248,1.2.643.5.1.13.13.12.2.77.8335,1.2.643.5.1.13.13.12.2.77.8335.0.117053,1.2.643.5.1.13.13.12.2.77.8335.0.117053.14199,55.884151,37.493506,гмосква,г,троицк,москва,NaN,NaN,NaN,NaN
701,1.2.643.5.1.13.13.12.2.93.79977,1.2.643.5.1.13.13.12.2.93.79977.0.670175,1.2.643.5.1.13.13.12.2.93.79977.0.670175.692776,48.071773,37.962265,республикадонецкаянародная,г,макеевка,no_data,NaN,NaN,NaN,NaN
702,1.2.643.5.1.13.13.12.2.93.79977,1.2.643.5.1.13.13.12.2.93.79977.0.670175,1.2.643.5.1.13.13.12.2.93.79977.0.670175.692770,48.071773,37.962265,республикадонецкаянародная,г,макеевка,no_data,NaN,NaN,NaN,NaN
703,1.2.643.5.1.13.13.12.2.93.79977,1.2.643.5.1.13.13.12.2.93.79977.0.670175,1.2.643.5.1.13.13.12.2.93.79977.0.670175.692763,48.071773,37.962265,республикадонецкаянародная,г,макеевка,no_data,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
617199,1.2.643.5.1.13.13.12.2.93.80007,1.2.643.5.1.13.13.12.2.93.80007.0.664272,1.2.643.5.1.13.13.12.2.93.80007.0.664272.637189,48.002628,37.766028,республикадонецкаянародная,г,донецк,no_data,NaN,NaN,NaN,NaN
617200,1.2.643.5.1.13.13.12.2.93.80007,1.2.643.5.1.13.13.12.2.93.80007.0.664272,1.2.643.5.1.13.13.12.2.93.80007.0.664272.637190,48.002628,37.766028,республикадонецкаянародная,г,донецк,no_data,NaN,NaN,NaN,NaN
617201,1.2.643.5.1.13.13.12.2.93.80007,1.2.643.5.1.13.13.12.2.93.80007.0.664272,1.2.643.5.1.13.13.12.2.93.80007.0.664272.637192,48.002628,37.766028,республикадонецкаянародная,г,донецк,no_data,NaN,NaN,NaN,NaN
617202,1.2.643.5.1.13.13.12.2.93.80007,1.2.643.5.1.13.13.12.2.93.80007.0.664272,1.2.643.5.1.13.13.12.2.93.80007.0.664272.637197,48.002628,37.766028,республикадонецкаянародная,г,донецк,no_data,NaN,NaN,NaN,NaN


In [88]:
df_adr_s[df_adr_s["reg"] == "no_data"]

,mo_oid,depart_oid,room_oid,lat,lon,reg,pref,area
616885,1.2.643.5.1.13.13.12.2.29.69422,no_data,no_data,no_data,no_data,no_data,no_data,no_data
617102,1.2.643.5.1.13.13.12.2.63.65862,no_data,no_data,no_data,no_data,no_data,no_data,no_data
617832,1.2.643.5.1.13.13.12.1.16.76554,no_data,no_data,no_data,no_data,no_data,no_data,no_data
618268,1.2.643.5.1.13.13.12.2.20.68280,no_data,no_data,no_data,no_data,no_data,no_data,no_data
618958,1.2.643.5.1.13.13.12.2.42.69020,no_data,no_data,no_data,no_data,no_data,no_data,no_data
...,...,...,...,...,...,...,...,...
904436,no_data,1.2.643.5.1.13.13.12.2.71.17001.0.449629,no_data,no_data,no_data,no_data,no_data,no_data
904439,no_data,1.2.643.5.1.13.13.12.2.86.20502.0.447860,no_data,no_data,no_data,no_data,no_data,no_data
904442,no_data,1.2.643.5.1.13.13.12.2.78.22275.0.471852,no_data,no_data,no_data,no_data,no_data,no_data
926168,no_data,1.2.643.5.1.13.13.12.2.23.47893.0.532540,no_data,no_data,no_data,no_data,no_data,no_data


In [87]:
df_reg = pd.DataFrame(df_adr_s["reg"]).drop_duplicates().reset_index(drop=True)
df_reg

,reg
0,кемеровская область
1,смоленская область
2,вологодская область
3,республика дагестан
4,московская область
...,...
111,алтай
112,чувашская республика - чувашия
113,город байконур
114,г. байконур
